Hemos creado un par de carpetas: una para grabar los audios y otra para las transcirpciones. Las trans ripciones las he guardado en formato JSON, pues parecía a priori lo mejor forma de guardarlo. Si embargo, creo que la siguiente prueba de concepto ha de tener en cuenta: 1. Crear fingerprints, 2. Guardar la transcruipcion en una tabla, en una base de datos de tipo PostgreSql.

In [3]:
import whisper
import subprocess
import os
from datetime import datetime
import json

def grabar_y_transcribir_continuamente(duracion_fragmento=60, duracion_total=300, carpeta_audio="grabaciones", 
                                       carpeta_transcripciones = "transcripciones"):
    """
    Graba audio en fragmentos y transcribe simultáneamente.
    """
    # Crear carpeta de salida si no existe
    # os.makedirs(carpeta_salida, exist_ok=True)
    os.makedirs(carpeta_audio, exist_ok=True)
    os.makedirs(carpeta_transcripciones, exist_ok=True)

    modelo = whisper.load_model("base")  # Carga el modelo de Whisper
    tiempo_inicial = datetime.now()

    tiempo_transcurrido = 0
    while tiempo_transcurrido < duracion_total:
        # Nombre del archivo basado en el timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archivo_audio = os.path.join(carpeta_audio, f"audio_{timestamp}.wav")
        archivo_transcripcion = os.path.join(carpeta_transcripciones, f"transcripcion_{timestamp}.json")

        # Grabar el fragmento con FFmpeg
        print(f"Grabando fragmento: {archivo_audio}")
        ffmpeg_command = [
            "ffmpeg", "-y", "-f", "dshow", "-i", "audio=Stereo Mix (Realtek(R) Audio)",
            "-t", str(duracion_fragmento), archivo_audio
        ]
        subprocess.run(ffmpeg_command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Fragmento guardado: {archivo_audio}")

        # Transcribir el fragmento
        print(f"Transcribiendo fragmento: {archivo_audio}")
        resultado = modelo.transcribe(archivo_audio, language="es", word_timestamps=True)
        transcripcion = [
            {"timestamp": segmento["start"], "text": segmento["text"]}
            for segmento in resultado["segments"]
        ]

        # Guardar la transcripción en JSON
        datos_guardar = {
            "metadata": {
                "archivo_audio": archivo_audio,
                "timestamp_inicio": timestamp,
                "duracion_fragmento": duracion_fragmento
            },
            "transcription": transcripcion
        }
        with open(archivo_transcripcion, "w", encoding="utf-8") as f:
            json.dump(datos_guardar, f, indent=4, ensure_ascii=False)

        print(f"Transcripción guardada en: {archivo_transcripcion}")

        # Actualizar tiempo transcurrido
        tiempo_transcurrido = (datetime.now() - tiempo_inicial).total_seconds()

    print("Grabación y transcripción completa.")




Grabando fragmento: grabaciones\audio_20241125_234837.wav
Fragmento guardado: grabaciones\audio_20241125_234837.wav
Transcribiendo fragmento: grabaciones\audio_20241125_234837.wav
Transcripción guardada en: transcripciones\transcripcion_20241125_234837.json
Grabando fragmento: grabaciones\audio_20241125_235000.wav
Fragmento guardado: grabaciones\audio_20241125_235000.wav
Transcribiendo fragmento: grabaciones\audio_20241125_235000.wav
Transcripción guardada en: transcripciones\transcripcion_20241125_235000.json
Grabando fragmento: grabaciones\audio_20241125_235119.wav
Fragmento guardado: grabaciones\audio_20241125_235119.wav
Transcribiendo fragmento: grabaciones\audio_20241125_235119.wav
Transcripción guardada en: transcripciones\transcripcion_20241125_235119.json
Grabación y transcripción completa.


In [ ]:
# Ejecutar
grabar_y_transcribir_continuamente(duracion_fragmento=60, duracion_total=180)